<a href="https://colab.research.google.com/github/AJamal27891/University-coursees-projects/blob/master/Albert_Matching_entities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
!rm -r  test_trainer 

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# download dependencies

In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 2.5MB 22.7MB/s 
     |████████████████████████████████| 901kB 49.2MB/s 
     |████████████████████████████████| 3.3MB 45.9MB/s 
     |████████████████████████████████| 245kB 35.8MB/s 
     |████████████████████████████████| 122kB 51.4MB/s 
     |████████████████████████████████| 245kB 51.5MB/s 


## load datasets 

In [ ]:
dirty_walmart_amazon = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_walmart_amazon'
dirty_dblp_scholar = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_dblp_scholar'
dirty_amazon_ituns = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_amazon_itunes'
dirty_dblp_acm = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_dblp_acm'
abt_buy = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/abt_buy'
amazon_google = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/amazon_google'
QQP= r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/QQP'

## Test dataset merger functions

In [ ]:
import pandas as pd 
data_A = pd.DataFrame({'id':[1,2,3,4,5,6],'price':[.2,.5,.6,.7,1,.8],'describtion':['samsung s5','iphone','xperia','sonny','hstc','xomi']})
data_B = pd.DataFrame({'id':[1,2,3,4,5,6],'price':[.2,.5,.6,.7,1,.8],'describtion':['iphone','xperia','sonny','huawi','xomi','samsung s5']})
lables = pd.DataFrame({'left_id':[1,2,3,4,5,6],'right_id':[6,1,2,3,4,5],'label':[1,1,1,1,0,1]})

In [ ]:
total_data = lables.merge(data_A,left_on='left_id',right_on='id')
total_data = total_data.merge(data_B,left_on='right_id',right_on='id')
total_data

,left_id,right_id,label,id_x,price_x,describtion_x,id_y,price_y,describtion_y
0,1,6,1,1,0.2,samsung s5,6,0.8,samsung s5
1,2,1,1,2,0.5,iphone,1,0.2,iphone
2,3,2,1,3,0.6,xperia,2,0.5,xperia
3,4,3,1,4,0.7,sonny,3,0.6,sonny
4,5,4,0,5,1.0,hstc,4,0.7,huawi
5,6,5,1,6,0.8,xomi,5,1.0,xomi


In [ ]:
def remove_id_columns(total_data):
  for x in total_data:
    if x.endswith('id'):
      total_data.drop(x,axis=1,inplace=True)
    if x.startswith('id'):
      total_data.drop(x,axis=1,inplace=True)
  return total_data  
remove_id_columns(total_data)

,label,price_x,describtion_x,price_y,describtion_y
0,1,0.2,samsung s5,0.8,samsung s5
1,1,0.5,iphone,0.2,iphone
2,1,0.6,xperia,0.5,xperia
3,1,0.7,sonny,0.6,sonny
4,0,1.0,hstc,0.7,huawi
5,1,0.8,xomi,1.0,xomi


In [ ]:
def remove_id_columns(total_data):
  for x in total_data:
    if x.endswith('id'):
      total_data.drop(x,axis=1,inplace=True)
    if x.startswith('id'):
      total_data.drop(x,axis=1,inplace=True)
  return total_data  
remove_id_columns(total_data)

def merger(data_A,data_B,train,test,valid,
           remove_ids=True,
           merge_attributes=True,
           drop_original=True,
           tuples=True):
  train_data = train.merge(data_A,left_on='ltable_id',right_on='id')
  train_data = train_data.merge(data_B,left_on='rtable_id',right_on='id')
  test_data = test.merge(data_A,left_on='ltable_id',right_on='id')
  test_data = test_data.merge(data_B,left_on='rtable_id',right_on='id')
  valid_data = valid.merge(data_A,left_on='ltable_id',right_on='id')
  valid_data = valid_data.merge(data_B,left_on='rtable_id',right_on='id')
  if remove_ids:
    train_data = remove_id_columns(train_data)
    test_data = remove_id_columns(test_data)
    valid_data = remove_id_columns(valid_data)
  if merge_attributes:
    train_data['text_left'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_x')]].apply(lambda x : '. [attribute] '.join(x.astype(str)) ,axis=1)
    train_data['text_right'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_y')]].apply(lambda x : '. [attribute] '.join(x.astype(str)) ,axis=1) 
    test_data['text_left'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_x')]].apply(lambda x : '. [attribute] '.join(x.astype(str)) ,axis=1)
    test_data['text_right'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_y')]].apply(lambda x : '. [attribute] '.join(x.astype(str)) ,axis=1) 
    valid_data['text_left'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_x')]].apply(lambda x : '. [attribute] '.join(x.astype(str)) ,axis=1)
    valid_data['text_right'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_y')]].apply(lambda x : '. [attribute] '.join(x.astype(str)) ,axis=1) 

  if drop_original:
    train_data= train_data[['label','text_left','text_right']]
    test_data= test_data[['label','text_left','text_right']]
    valid_data= valid_data[['label','text_left','text_right']]
  if tuples:
    train_data['tuples']= zip(train_data['text_left'],train_data['text_right'])
    test_data['tuples']= zip(test_data['text_left'],test_data['text_right'])
    valid_data['tuples']= zip(valid_data['text_left'],valid_data['text_right'])

  return train_data, test_data, valid_data

# Data Preprocessing

## Amazon Google data

In [ ]:
table_A = pd.read_csv(amazon_google+r'/deep_matcher/tableA.csv')
table_B = pd.read_csv(amazon_google+r'/deep_matcher/tableB.csv')
AG_train = pd.read_csv(amazon_google+r'/deep_matcher/train.csv')
AG_test = pd.read_csv(amazon_google+r'/deep_matcher/test.csv')
AG_valid = pd.read_csv(amazon_google+r'/deep_matcher/valid.csv')
AG_train, AG_test, AG_valid = merger(table_A,table_B,AG_train,AG_test,AG_valid)
AG_train.head()

In [ ]:
import os 
try:
  os.mkdir('merged_data')
  os.mkdir('merged_data/amazon_google')
except FileExistsError:
  pass

AG_train.to_csv(r'merged_data/amazon_google/train.csv')
AG_test.to_csv(r'merged_data/amazon_google/test.csv')
AG_valid.to_csv(r'merged_data/amazon_google/valid.csv')



In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset('csv',data_files={'train':'merged_data/amazon_google/train.csv',
                                              'test':'merged_data/amazon_google/test.csv',
                                              'valid':'merged_data/amazon_google/valid.csv'})

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")

In [ ]:
def tokenize_function(examples):
    # examples['text_left'].head()
    # examples['tuples'] = examples[["text_left","text_right"]].apply(lambda x: (x[0],x[1]),axis=1)
    return tokenizer(examples['tuples'], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns('Unnamed: 0')
tokenized_datasets

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_datasets['train'], eval_dataset=tokenized_datasets['test']
)

In [ ]:
trainer.train()

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)
trainer.evaluate()

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: tuples, text_left, text_right.
***** Running Evaluation *****
  Num examples = 2293
  Batch size = 8


{'eval_f1': 0.07887323943661972,
 'eval_loss': 0.4438517689704895,
 'eval_runtime': 20.6975,
 'eval_samples_per_second': 110.786,
 'eval_steps_per_second': 13.866}